In [1]:
import pandas as pd
import numpy as np
import pickle
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow import keras
import keras_nlp
import platform
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

print(f"TensorFlow Version: {tf.__version__}")
print(f"KerasNLP Version: {keras_nlp.__version__}")
print(f"Python Version: {platform.python_version()}")

# Recommended: TensorFlow >= 2.15, KerasNLP >= 0.8

2025-12-02 09:42:48.444431: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-02 09:42:48.465784: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-02 09:42:48.638647: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-02 09:42:48.787956: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764639768.913838    1137 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764639768.95

TensorFlow Version: 2.19.1
KerasNLP Version: 0.23.0
Python Version: 3.12.3


In [7]:
# Check for GPU devices
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("GPU Detected:", gpus)
    # If the GPU is detected and TensorFlow is configured correctly,
    # cuDNN is almost certainly working and enabled.
    print("cuDNN status: Inferred as ENABLED and operational.")
else:
    print("No GPU detected. cuDNN is not being used.")

No GPU detected. cuDNN is not being used.


In [19]:
df = pd.read_csv('data/csv_training/training_course_directory.csv')
df.sample(n=2)

,coursetitle,trainingprovideralias,ihl_status,courseratings_stars,jobcareer_impact_stars,full_course_fee,course_fee_after_subsidies,number_of_hours,training_commitment,about_this_course,what_you_learn,attendancecount
14319,"Professional, Legal and Ethical Healthcare Pra...",HMI INSTITUTE OF HEALTH SCIENCES PTE. LTD.,NON-IHL,0.0,0.0,902.0,451.0,41.0,Full Time,Enable learner to apply organisational code of...,Learning unit 1: Concept of accountability as ...,143.0
12041,Principles of Management (in MC in Business Fu...,Temasek Polytechnic,IHL,0.0,0.0,1698.0,1698.0,60.0,Part Time,"If you attend this course, you will be equippe...",This subject covers the key management functio...,0.0


In [20]:
df.columns.tolist()

['coursetitle',
 'trainingprovideralias',
 'ihl_status',
 'courseratings_stars',
 'jobcareer_impact_stars',
 'full_course_fee',
 'course_fee_after_subsidies',
 'number_of_hours',
 'training_commitment',
 'about_this_course',
 'what_you_learn',
 'attendancecount']

In [21]:
# Combine all descriptive columns (A-E) into a single text feature
# This embeds the structured data (Rating, Fee, etc.) directly into the text for BERT
df['Full_Text'] = (
    "TITLE: " + df['coursetitle'].astype(str) + ". " +
    "SCHOOL: " + df['ihl_status'].astype(str) + ". " +
    "COURSE RATING: " + df['courseratings_stars'].astype(str) + ". " +
    "JOB RATING: " + df['jobcareer_impact_stars'].astype(str) + ". " +
    "FEE: $" + df['full_course_fee'].astype(str) + ". " +
    "FEE AFTER SUBSIDIES: $ " + df['course_fee_after_subsidies'].astype(str) + ". " +
    "TRAINING HOUR: " + df['number_of_hours'].astype(str) + " hours. " +
    "TRAINING COMMITMENT: " + df['training_commitment'].astype(str) + ". " +
    "DESCRIPTIONS: " + df['about_this_course'].astype(str) + ". " +   
    "SKILLS: " + df['what_you_learn'].astype(str)
)

X = df['Full_Text'].values
y = df['attendancecount'].values

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Important: Scale the target variable (y) for better regression performance
scaler = StandardScaler()
y_train_scaled = scaler.fit_transform(y_train.reshape(-1, 1))
y_test_scaled = scaler.transform(y_test.reshape(-1, 1))

print(f"Sample Input Text:\n{X_train[0]}")
print(f"Sample Target Value (Original): {y_train[0]}")
print(f"Sample Target Value (Scaled): {y_train_scaled[0]}")


Sample Input Text:
TITLE: Java Programming for Complete Beginners(Classroom and Synchronous e-learning). SCHOOL: NON-IHL. COURSE RATING: 4.0. JOB RATING: 0.0. FEE: $1400.0. FEE AFTER SUBSIDIES: $ 420.0. TRAINING HOUR: 28.0 hours. TRAINING COMMITMENT: Part Time. DESCRIPTIONS: LO1: Gather and confi rm the architecture requirements_x000D_LO2: Derive high-level design from specifi cation_x000D_LO3: Apply Java language syntax and layout_x000D_LO4: Apply basic object-oriented interaction principles_x000D_LO5: Document the design. SKILLS: Object-oriented design is a process by which a set of detailed object-oriented software design models are built, which are then used by the programmers to write and test programs for the new system. Systems design is the bridge between user requirements and programming the new system. One strength of the object-oriented approach is that the design models are often an extension of the requirements models. Object-oriented design is an analytical, rigorous, and

In [22]:
def build_keras_nlp_regression_model():
    # Use the BertClassifier Task Model for simpler setup
    # num_classes=1 gives us one output neuron
    # change from bert_base_en_uncased (110million params) to smaller model
    # keras_nlp.models.BertClassifier.from_preset() will insert [CLS] and [SEP] tokens automatically
    # bert_small_en_uncased (29 million params), bert_mini_en_uncased (14million params) or bert_tiny_en_uncased(4.4million params)
    model = keras_nlp.models.BertClassifier.from_preset(
        "bert_tiny_en_uncased",
        num_classes=1, 
        load_weights=True,
        # Set max sequence length based on your average text length
        # Change from 128 to 64 as WSL keep crashing during training
        preprocessor_config={"sequence_length": 128} 
    )
    
    # Critical modification: change the final layer's activation from softmax/sigmoid to linear
    # This converts the model from classification to regression.
    model.layers[-1].activation = tf.keras.activations.linear
    
    # Compile for Regression
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5), # Low learning rate for fine-tuning
        loss=tf.keras.losses.MeanSquaredError(), # MSE loss for regression
        metrics=[tf.keras.metrics.MeanAbsoluteError(name='mae')]
    )
    
    return model

# Create the model
regression_model = build_keras_nlp_regression_model()
print("\nModel Summary:")
regression_model.summary()


Model Summary:


Preprocessor: "bert_text_classifier_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ bert_tokenizer (BertTokenizer)                                │                       Vocab size: 30,522 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "bert_text_classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ segment_ids (InputLayer)      │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bert_backbone (BertBackbone)  │ [(None, 128), (None,      │       4,385,920 │ padding_mask[0][0],        │
│                               │ None, 128)]               │                 │ segment_ids[0][0],         │
│                               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ classifier_dropout (Dropout)  │ (None, 128)               │               0 │ bert_backbone[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ logits (Dense)                │ (None, 1)                 │             129 │ classifier_dropout[0][0]   │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 4,386,049 (16.73 MB)

 Trainable params: 4,386,049 (16.73 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
# --- Configuration ---
PRESET = "bert_tiny_en_uncased"
MAX_SEQUENCE_LENGTH = 128 # Your limit

# --- 1. Load the Tokenizer ---
# We use the raw tokenizer to see the length BEFORE padding/truncation
tokenizer = keras_nlp.models.BertTokenizer.from_preset(
    PRESET
)

# --- 2. Define a function to calculate token length ---
# This function calculates the number of tokens for a given text,
# *including* the automatically added [CLS] and [SEP] tokens.
def get_token_length(text):
    # Tokenize the text.
    token_ids = tokenizer.tokenize(text)
    # The length of the resulting tensor is the number of tokens
    return tf.shape(token_ids)[0].numpy()

# --- 3. Apply the function to your full dataset (X_train) ---
print("Calculating token lengths...")
# Use list comprehension for efficient processing
token_lengths = [get_token_length(text) for text in X_train]

# Convert to a numpy array for easy statistical calculation
token_lengths = np.array(token_lengths)

# --- 4. Analyze Results ---

# Calculate statistics
avg_length = np.mean(token_lengths)
max_length = np.max(token_lengths)
min_length = np.min(token_lengths)

# Calculate truncation metrics
# An input is truncated if its length is greater than the MAX_SEQUENCE_LENGTH
truncated_count = np.sum(token_lengths > MAX_SEQUENCE_LENGTH)
total_samples = len(X_train)
percent_truncated = (truncated_count / total_samples) * 100

# --- 5. Print Summary ---
print("\n--- Token Length Analysis Summary ---")
print(f"BERT Preset Used: {PRESET}")
print(f"Max Sequence Length in Model: {MAX_SEQUENCE_LENGTH}")
print("-" * 40)
print(f"Total Training Samples: {total_samples}")
print(f"Average Token Length: {avg_length:.2f} tokens")
print(f"Maximum Token Length: {max_length} tokens")
print(f"Minimum Token Length: {min_length} tokens")
print("-" * 40)
print(f"Samples Exceeding {MAX_SEQUENCE_LENGTH} Tokens (Truncated): {truncated_count} samples")
print(f"Percentage of Truncated Samples: {percent_truncated:.2f}%")
print("-" * 40)

# Example of a long and short text's length
long_sample_index = np.argmax(token_lengths)
short_sample_index = np.argmin(token_lengths)

print(f"Length of Longest Sample: {token_lengths[long_sample_index]}")
print(f"Length of Shortest Sample: {token_lengths[short_sample_index]}")

# Note: The calculation above includes the [CLS] and [SEP] tokens.

Calculating token lengths...

--- Token Length Analysis Summary ---
BERT Preset Used: bert_tiny_en_uncased
Max Sequence Length in Model: 128
----------------------------------------
Total Training Samples: 20650
Average Token Length: 296.29 tokens
Maximum Token Length: 1426 tokens
Minimum Token Length: 66 tokens
----------------------------------------
Samples Exceeding 128 Tokens (Truncated): 19580 samples
Percentage of Truncated Samples: 94.82%
----------------------------------------
Length of Longest Sample: 1426
Length of Shortest Sample: 66


In [14]:
early_stopping = EarlyStopping(
    monitor='val_mae', 
    patience=1, 
    restore_best_weights=True,
    verbose=1 
)

history = regression_model.fit(
    x=X_train,
    y=y_train_scaled,
    validation_data=(X_test, y_test_scaled),
    epochs=3,
    batch_size=4, #change from 16 to 4 as WSL keep crashing during training
    callbacks=[early_stopping]
)

# Make predictions on the test set (results are scaled)
y_pred_scaled = regression_model.predict(X_test)

# Inverse transform the predictions to the original student count scale

y_pred = scaler.inverse_transform(y_pred_scaled)

# Display results
results_df = pd.DataFrame({
    'Actual_Students': y_test,
    'Predicted_Students': y_pred.flatten().round(0) # Round to nearest whole number
})

print("\n--- Sample Predictions (Test Set) ---")
print(results_df)

# Calculate final unscaled MAE
final_mae = np.mean(np.abs(results_df['Actual_Students'] - results_df['Predicted_Students']))
print(f"\nFinal Mean Absolute Error (Unscaled): {final_mae:.2f} students")

Epoch 1/3
5163/5163 ━━━━━━━━━━━━━━━━━━━━ 446s 86ms/step - loss: 0.9175 - mae: 0.2478 - val_loss: 0.7531 - val_mae: 0.2040
Epoch 2/3
5163/5163 ━━━━━━━━━━━━━━━━━━━━ 444s 86ms/step - loss: 0.8399 - mae: 0.2306 - val_loss: 0.7392 - val_mae: 0.2279
Epoch 2: early stopping
Restoring model weights from the end of the best epoch: 1.
162/162 ━━━━━━━━━━━━━━━━━━━━ 23s 136ms/step

--- Sample Predictions (Test Set) ---
      Actual_Students  Predicted_Students
0                 0.0                16.0
1                 0.0                12.0
2              1371.0               797.0
3               436.0               176.0
4                32.0                70.0
...               ...                 ...
5158             27.0               143.0
5159              5.0                10.0
5160             22.0               338.0
5161             13.0                45.0
5162              0.0                17.0

[5163 rows x 2 columns]

Final Mean Absolute Error (Unscaled): 149.14 students


In [16]:
# R2 Score (Coefficient of Determination)
# Best value is 1.0. Can be negative.
r2 = r2_score(y_test, y_pred)
print(f"R2 Score: {r2:.4f}")

# Mean Absolute Error (MAE)
# Measures the average magnitude of the errors in a set of predictions.
# Best value is 0.0.
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error (MAE): {mae:.4f}")

# Mean Squared Error (MSE)
# Measures the average of the squares of the errors. Penalizes larger errors more.
# Best value is 0.0.
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse:.4f}")

R2 Score: 0.1608
Mean Absolute Error (MAE): 149.1388
Mean Squared Error (MSE): 402566.8642


In [ ]:
#Save the BERT mode.
# MODEL_SAVE_PATH = 'data/model/bert_regression_model.keras'
# regression_model.save(MODEL_SAVE_PATH) 
# print(f"\nModel saved successfully to: {MODEL_SAVE_PATH}")


Model saved successfully to: data/model/bert_regression_model.keras


In [ ]:
# #Save the Fitted Scaler ---
# scaler_filename = 'data/model/bert_model_scaler.pkl'

# try:
#     with open(scaler_filename, 'wb') as file:
#         # Save the fitted scaler object
#         pickle.dump(scaler, file)
#     print(f"Scaler successfully saved as '{scaler_filename}'")
# except Exception as e:
#     print(f"Error saving scaler: {e}")

Scaler successfully saved as 'data/model/bert_model_scaler.pkl'


<h3>Load the model</h3>

In [2]:
#Load BERT MODEL
model_path = 'data/model/bert_regression_model.keras' 
loaded_model = keras.models.load_model(model_path)

E0000 00:00:1764639785.558263    1137 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1764639785.560124    1137 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/lowboonsiong/projects/bert_venv/lib/python3.12/site-packages/keras/src/saving/serialization_lib.py:749: UserWarning: `compile()` was not called as part of model loading because the model's `compile()` method is custom. All subclassed Models that have `compile()` overridden should also override `get_compile_config()` and `compile_from_config(config)`. Alternatively, you can call `compile()` 

In [3]:
#Load Scaler pickle model ---
scaler_filename = 'data/model/bert_model_scaler.pkl'

try:
    with open(scaler_filename, 'rb') as file:
        # Load the fitted scaler object
        loaded_scaler = pickle.load(file)
    print(f"Scaler successfully loaded from '{scaler_filename}'")

    # Verify that the loaded scaler has the correct values
    print(f"Loaded Scaler Mean: {loaded_scaler.mean_[0]:.2f}")
    print(f"Loaded Scaler Std Dev: {loaded_scaler.scale_[0]:.2f}")

except FileNotFoundError:
    print(f"Error: Scaler file '{scaler_filename}' not found.")
    loaded_scaler = None
except Exception as e:
    print(f"Error loading scaler: {e}")

Scaler successfully loaded from 'data/model/bert_model_scaler.pkl'
Loaded Scaler Mean: 139.53
Loaded Scaler Std Dev: 731.14


In [ ]:
def predict_new_course_students(model, scaler, course_data):
    #Remeber to change the key to new one after testing the first model
    new_input_text = (
        "TITLE: " + str(course_data['coursetitle']) + ". " +
        "SCHOOL: " + str(course_data['ihl_status']) + ". " +
        "COURSE RATING: " + str(course_data['courseratings_stars']) + ". " +
        "JOB RATING: " + str(course_data['jobcareer_impact_stars']) + ". " +
        "FEE: $" + str(course_data['full_course_fee']) + ". " +
        "FEE AFTER SUBSIDIES (IF ANY): $ " + str(course_data['course_fee_after_subsidies']) + ". " +
        "TRAINING HOUR: " + str(course_data['number_of_hours']) + ". " +
        "TRAINING COMMITMENT: " + str(course_data['training_commitment']) + ". " +
        "DESCRIPTIONS: " + str(course_data['about_this_course']) + ". " + 
        "SKILLS: " + str(course_data['what_you_learn'])
    )

    # Convert the single string into a NumPy array of strings for the model
    X_new = np.array([new_input_text])    
    y_pred_scaled = model.predict(X_new)
    
    # Inverse Transform and Finalize ---
    # Convert scaled value back to original student count
    y_pred = scaler.inverse_transform(y_pred_scaled)
    
    # Round to the nearest whole number and ensure it's not negative
    predicted_students = max(0, int(round(y_pred[0][0])))
    
    return predicted_students, new_input_text

# -----------------------------------------------------------------------------
## 🎯 Define the New Course Data
# -----------------------------------------------------------------------------

# Define the features for a hypothetical new course
new_course_features = {
    'coursetitle': "Engineering Ethics",
    'ihl_status': "IHL",
    'courseratings_stars': 0.0,
    'jobcareer_impact_stars': 0.0,
    'full_course_fee': 1461.0,
    'course_fee_after_subsidies': 438.30,
    'number_of_hours': 36,
    'training_commitment': "Full Time",
    'about_this_course': "Nowadays, technology has a pervasive and profound impact on everyday life, where engineers play a crucial role in its development. It is therefore extremely important that engineers understand the importance of safety, health, and welfare of the public, when developing this technology. They must be morally committed and equipped to tackle any ethical issues they may encounter. This course aims at training the student to reach such a status through discussions and typical case studies where real examples are thoroughly discussed.",
    'what_you_learn': "Ethics and Professionalism. Moral Choices and Ethical Dilemmas. Codes of Ethics. Moral Frameworks. Ethics as Social Experimentation. Safety and Risk. Assessing and Reducing Risk. Workplace Responsibilities and Rights. Truth and Truthfulness. Computer Ethics. Environmental Ethics;"
}

new_course_features2 = {
    #'coursetitle': "Public Areas Housekeeping Operations Management - L1 (Classroom)",
    'coursetitle': "Washing and Cleaning of Public Toilets, Showers and Change Rooms",
    'ihl_status': "IHL",
    'courseratings_stars': 3.0,
    'jobcareer_impact_stars': 3.0,
    'full_course_fee': 400.0,
    'course_fee_after_subsidies': 100.00,
    'number_of_hours': 16,
    'training_commitment': "Full Time",
    # 'about_this_course': "Nothing",
    # 'what_you_learn': "Nothing"
    'about_this_course': "Learner will have knowledge and application skills in public toilets washing and cleaning, showers and change rooms and be able to apply them to the workplace",
    'what_you_learn': "1. Following procedures for starting and ending shift. 2. following procedures for servicing and washing toilets, showers and change rooms 3. cleaning surfaces in toilets, showers and change rooms"
}

# --- Execute the Prediction ---
if loaded_model:
    predicted_count, input_text = predict_new_course_students(loaded_model, loaded_scaler, new_course_features2)

    print("\n--- NEW COURSE PREDICTION ---")
    print(f"Predicted Number of Students to Attend: **{predicted_count}**")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 557ms/step

--- NEW COURSE PREDICTION ---
Predicted Number of Students to Attend: **612**
